![title](https://ucm.es/themes/ucm16/media/img/logo.png)

# Bases de datos NoSQL

### Práctica Redis. Servicios de publicación/subscripción 

#### Parte 2. Subscripción

Lo primero, el nombre:

Daniel Candil Vizcaino

Ahora conectamos a Redis. Es importante que cambieis los datos por los de vuestra cuenta, para no saturar el servidor. Además, los datos deben ser los mismos del publisher, si no no funcionará

In [2]:
# Ojo, cambiar estos datos por los de vuestro acceso a red en redislabs
redisconexion = "redis-10465.c226.eu-west-1-3.ec2.cloud.redislabs.com"
redispuerto = 10465
passwd = "ae7jbWPnnrNuLcKFamKyb6IQCvj3ycse"

import redis
r = redis.Redis(host=redisconexion, password=passwd, port=redispuerto, charset="utf-8", decode_responses=True, db=0)
r.ping() # debe mostrar True

True

#### Pregunta 2: 1 punto

Declarar 3 subcriptores, s1, s2 y s3:
    
* s1 estará suscrito solo a 'news-money'

* s2 estará suscrito solo a 'news-selected'

* s3 estará suscrito solo a 'news-selected' y a 'news-other'

In [3]:
# solución
s1 = r.pubsub()  
s2 = r.pubsub()
s3 = r.pubsub()
s1.subscribe("news-money")
s2.subscribe("news-selected")
s3.subscribe("news-selected")
s3.subscribe("news-other")

#### Pregunta 3: 2 puntos

Tras ejecutar el código anterior, ejecutar el publisher, que tardará unos segundos (puede que un minuto). Ahora ya tenemos tanto subcriptores (s1,s2,s3) como mensajes publicados. Para probarlo, escribir una función muestra que imprime la clave 'data' y 'chanel' de los n primeros mensajes  del subscriptor de tipo (type) `message` o `pmessage`.

Nota: suponer que hay n mensajes, no hace falta comprobarlo

In [74]:
# solución
import json
# s es un subcriptor, n el número de mensajes a mostrar
def muestra(s,n):
    for x in range(n):
        getmsg = s.get_message()
        getmsg = json.dumps(getmsg)
        getmsg = json.loads(getmsg)
        canal = getmsg['channel']
        titular = getmsg['data']
        print("Mensaje  " + str((x+1)) + "  Canal  " + canal)
        print("Titular: " + str(titular) + '\n')

Para probar el código anterior

In [75]:
print("s1")
muestra(s1,3)
print("s2")
muestra(s2,3)
print("s3")
muestra(s3,3)

s1
Mensaje  1  Canal  news-money
Titular: Bengaluru-based bike taxi app Rapido raises ₹52 crore

Mensaje  2  Canal  news-money
Titular: IIT Delhi alumnus' lingerie brand Clovia raises $10 million

Mensaje  3  Canal  news-money
Titular: Farm loan waiver of ₹10, ₹20 a cruel joke: Shivraj Chouhan

s2
Mensaje  1  Canal  news-selected
Titular: Strongly support India's policy response to COVID-19 pandemic: IMF

Mensaje  2  Canal  news-selected
Titular: China sends 6.5 lakh test kits to India, to send 20 lakh kits in next 15 days

Mensaje  3  Canal  news-selected
Titular: Trump halting WHO funding doesn't make any sense right now: Melinda Gates

s3
Mensaje  1  Canal  news-selected
Titular: 1

Mensaje  2  Canal  news-other
Titular: 2

Mensaje  3  Canal  news-other
Titular: My wife hates it when I work from home: Banksy shares graffiti from bathroom



La salida dependerá de las noticias, pero debe seguir el formato:

    s1
    Mensaje  1  Canal  news-money 
    Titular: Maharashtra govt to distribute ₹2,900 cr for drought relief 

    Mensaje  2  Canal  news-money 
    Titular: Fresh milk startup Country Delight raises $7-10 mn: Reports 

    Mensaje  3  Canal  news-money 
    Titular: Deals worth ₹3 lakh cr signed at Global Investors Meet: TN 

    s2
    Mensaje  1  Canal  news-selected 
    Titular: Came back from India trip, amazed at the changes: Mark Mobius 
    ....

#### Pregunta 4: 1 punto

* Crear un nuevo subscriptor s4
* Utilizar la subscripción por patrones para subscribirlo a todos los canales que empiezan por `news-`


In [76]:
s4 = r.pubsub()  
s4.psubscribe('news-*')

Para probar:

In [77]:
# antes de ejecutar este código ejecutar de nuevo el publisher (la última caja de código)
muestra(s4,4)

Mensaje  1  Canal  news-*
Titular: 1

Mensaje  2  Canal  news-other
Titular: Rangoli's Twitter account suspended over controversial tweet on attack on UP doctors

Mensaje  3  Canal  news-other
Titular: We're not mad: Shatrughan after director shared Sonakshi's old pic at shoot amid lockdown

Mensaje  4  Canal  news-other
Titular: IPL 2020 suspended till further notice amid coronavirus pandemic



#### Pregunta 5: 2 puntos
Lo normal es que no queramos mostrar la información sin más por pantalla, sino que queramos grabar lo publicado en una base de datos. En nuestro caso se pide:

* Crear un servidor de mongo (o conectar a Atlas, como se desee)
* Escribir una función graba(s,db) que grabe cada mensaje en una colección con el mismo nombre que el canal en el que se ha publicado el mensaje. Cada documento solo tendrá una clave `titular` (aparte del _id, que dejaremos que se genera automáticamente)

graba(s,db) grabará en mongo titulares hasta que la función `get_message` devuelva None

**AViso**: el siguiente código borra la base de datos news, asegurarse de no tener nada importante

In [101]:
# iniciamos mongo
from pymongo import MongoClient
client = MongoClient('mongodb://127.0.0.1:27017/')

## OJO: borramos la base de datos news
client.drop_database('news')
db = client.news

# graba todos los mensajes del publisher s en la base de datos db, con el nombre
# de colecciones que indice 'channel'
def graba(s,db):
    noEncontrado = True
    while noEncontrado:
        getmsg = s.get_message()
        noEncontrado = getmsg != None
        if noEncontrado == False:
            break
        getmsg = json.dumps(getmsg)
        getmsg = json.loads(getmsg)
        coleccion = getmsg['channel']
        titular = getmsg['data']
        db[coleccion].insert_one({"titular":titular})

Para probarlos nos declaramos un subcriptor

In [102]:

s5 = r.pubsub() 
s5.subscribe("news-selected")
s5.subscribe("news-money")


Ahora ejecutamos el publisher, y a continuación la llamada a nuestra función, que grabará en mongo en dos colecciones, news-selected, y news-money

In [103]:
graba(s5,db)

{'type': 'subscribe', 'pattern': None, 'channel': 'news-selected', 'data': 1}
{'type': 'subscribe', 'pattern': None, 'channel': 'news-money', 'data': 2}
{'type': 'message', 'pattern': None, 'channel': 'news-selected', 'data': "Strongly support India's policy response to COVID-19 pandemic: IMF"}
{'type': 'message', 'pattern': None, 'channel': 'news-selected', 'data': 'China sends 6.5 lakh test kits to India, to send 20 lakh kits in next 15 days'}
{'type': 'message', 'pattern': None, 'channel': 'news-money', 'data': 'Fintech startup Setu raises $15M led by Lightspeed, Falcon Edge'}
{'type': 'message', 'pattern': None, 'channel': 'news-selected', 'data': 'Once you win in Indian market, you can win anywhere: Sequoia Capital India MD'}
{'type': 'message', 'pattern': None, 'channel': 'news-money', 'data': "Nirav Modi's ₹100-crore sea-facing Alibaug bungalow being demolished"}
{'type': 'message', 'pattern': None, 'channel': 'news-money', 'data': 'Zee firms lose ₹14,000 crore market value in a

Vamos a ver si se ha insertado bien mirando los primeros documentos de cada coleccion

In [104]:
def muestra_primeros(db,n):
    for c in db.list_collection_names():
        print("Colección ",c)
        for doc in db[c].find({},{"_id":0}).limit(n):
            print(doc)
       

In [105]:
muestra_primeros(db,10)

Colección  news-money
{'titular': 2}
{'titular': 'Fintech startup Setu raises $15M led by Lightspeed, Falcon Edge'}
{'titular': "Nirav Modi's ₹100-crore sea-facing Alibaug bungalow being demolished"}
{'titular': 'Zee firms lose ₹14,000 crore market value in a day after media report'}
{'titular': 'Bengaluru-based bike taxi app Rapido raises ₹52 crore'}
{'titular': "IIT Delhi alumnus' lingerie brand Clovia raises $10 million"}
{'titular': 'Farm loan waiver of ₹10, ₹20 a cruel joke: Shivraj Chouhan'}
Colección  news-selected
{'titular': 1}
{'titular': "Strongly support India's policy response to COVID-19 pandemic: IMF"}
{'titular': 'China sends 6.5 lakh test kits to India, to send 20 lakh kits in next 15 days'}
{'titular': 'Once you win in Indian market, you can win anywhere: Sequoia Capital India MD'}
{'titular': "'Roma' actor may miss Oscars after US visa was denied thrice"}
{'titular': "India's pluralism is its greatest strength: President Kovind"}
{'titular': 'Resolve issue through di

La salida debe tener el aspecto:    
    
    Colección  news-selected
    {'titular': 'US govt agency condemns Pak as food aid being denied to Hindus amid coronavirus'}
    {'titular': 'Coronaviruses found in two bat species in India: ICMR'}
    ....